In [1]:
import pygdf
import numpy as np

In [2]:
pygdf.DataFrame

pygdf.dataframe.DataFrame

In [3]:
df = pygdf.DataFrame()
nelem = 10**6  # A million item
df['key1'] = np.random.randint(0, 5, nelem)
df['key2'] = np.random.randint(0, 3, nelem)
df['val1'] = np.arange(1, 1 + nelem)
df['val2'] = np.random.random(nelem)

In [4]:
df.head().to_pandas()

,key1,key2,val1,val2
0,3,1,1,0.971857
1,0,0,2,0.835899
2,2,2,3,0.006712
3,2,0,4,0.088507
4,4,1,5,0.232680


In [5]:
df.groupby(by=['key1', 'key2']).mean()

/home/ajay/anaconda3/lib/python3.5/site-packages/numba/cuda/compiler.py:234: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


GDFError: GDF_CUDA_ERROR